In [ ]:
"""
#CURRENT VERSION 
A simple test of Ray
This example uses placement_group API to spread work around
"""
import random
import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self, actor_id) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()
        self.actor_id = actor_id
        
    def ping(self):
        print(f"{self.actor_id} {self.pid} {self.hostname} {self.ip} {time.time()} - ping")
        time.sleep(random.randint(1,3))
        return f"{self.actor_id}"

@ray.remote
def main():
# Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")
# Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    time.sleep(1)
    
# Ping-Pong test
    messages = [actors[a].ping.remote() for a in actors]
    time.sleep(1)
    for _ in range(20):
        new_messages, messages = ray.wait(messages, num_returns=1)
        for ray_message_id in new_messages:
            pong = ray.get(ray_message_id)
            print(pong, "- pong")
            position = key_list.index(pong[0]+pong[5])
            actor = value_list[position]
            check = actor.ping.remote()
            messages.append(check)
            time.sleep(1)

actors = {
    "a1" : Actor.remote(actor_id="actor1"),
    "a2" : Actor.remote(actor_id="actor2"),
    "a3" : Actor.remote(actor_id="actor3"),
    "a4" : Actor.remote(actor_id="actor4"),
    "a5" : Actor.remote(actor_id="actor5")
}
key_list = list(actors.keys())
value_list = list(actors.values())

print(actors)
if __name__ == "__main__":
    main.remote()

In [ ]:
# VERSION TWO
"""
A simple test of Ray
This example uses placement_group API to spread work around
"""

import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()

    def ping(self):
        
        print(f"{self.pid} {self.hostname} {self.ip} - ping")
        time.sleep(1)
        return f"{self.pid} {self.hostname} {self.ip} - pong"

@ray.remote
def main():
   
    # Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")

    # Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    actor = []
    for a in actors:
        node_ip_str = f"node:{ray._private.services.get_node_ip_address()}"
        actor.append(Actor.remote())
        time.sleep(1)

    # Ping-Pong test
    for _ in range(2):
        for a in actors:
            time.sleep(1)
            messages = a.ping.remote()
            time.sleep(1)
            print(f"Received back message {ray.get(messages)} \n")
        

            
actors = [Actor.remote() for _ in range(5)]
print(actors)

if __name__ == "__main__":
    main.remote()

In [ ]:
# VERSION THREE
"""
A simple test of Ray
This example uses placement_group API to spread work around
"""

import os
import platform
import ray
import time
ray.init(ignore_reinit_error=True)

@ray.remote
class Actor():
    def __init__(self) -> None:
        self.pid = os.getpid()
        self.hostname = platform.node()
        self.ip = ray._private.services.get_node_ip_address()

    def ping(self):
        
        print(f"{self.pid} {self.hostname} {self.ip} - ping")
        time.sleep(1)
        return f"{self.pid} {self.hostname} {self.ip} - pong"

@ray.remote
def main():
   
    # Get list of nodes to use
    print(f"Found {len(actors)} Worker nodes in the Ray Cluster:")

    # Setup one Actor per node
    print(f"Setting up {len(actors)} Actors...")
    actor = []
    for a in actors:
        node_ip_str = f"node:{ray._private.services.get_node_ip_address()}"
        actor.append(Actor.remote())
        time.sleep(1)

    # Ping-Pong test
    for _ in range(2):
        messages = [a.ping.remote() for a in actors]
        for msg in ray.get(messages):
            time.sleep(1)
            print(f"Received back message {msg}")
            time.sleep(1)

            
actors = [Actor.remote() for _ in range(5)]
print(actors)

if __name__ == "__main__":
    main.remote()